In [21]:
import pandas as pd
from tqdm import tqdm
from rapidfuzz import process, fuzz
from concurrent.futures import ThreadPoolExecutor

# Load the disease dataset
# df_diseases = pd.read_csv("../CTD/data/target_disease/sample_cor_99.csv")
df_diseases = pd.read_csv("../CTD/data/target_disease/sample_cor_99.csv")

diseases = [d.replace("_", " ").lower() for d in df_diseases.columns.tolist()]

# Load the ICD-10 dataset
df_icd = pd.read_csv("data/DiagAll_Eng.csv")
df_icd = df_icd.drop_duplicates(subset=["ShortDescription"])  # Ensure unique descriptions
icd_dict = dict(zip(df_icd["ShortDescription"].str.lower(), df_icd["Code"]))
icd_descriptions = set(icd_dict.keys())  # Convert to set for faster lookup

def map_disease(disease_name):
    # Check for exact match first
    if disease_name in icd_descriptions:
        return disease_name
    
    # If no exact match, use faster fuzzy string matching
    best_match = process.extractOne(disease_name, icd_descriptions, scorer=fuzz.WRatio)
    return best_match[0] if best_match and best_match[1] > 60 else "UNKNOWN"

# Use multithreading to speed up mapping (max 6 threads)
with ThreadPoolExecutor(max_workers=6) as executor:
    results = list(tqdm(executor.map(map_disease, diseases), total=len(diseases), desc="Mapping diseases to ICD-10 codes"))

diseases_mapping = dict(zip(diseases, results))

# Print each mapping on a new line
for disease, icd in list(diseases_mapping.items())[:50]:
    print(f"{disease}: {icd}")


Mapping diseases to ICD-10 codes: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17701/17701 [00:16<00:00, 1042.04it/s]

synucleinopathy: healthy
trypanosomiasis: african trypanosomiasis
giardiasis: filariasis
pulmonary sarcoidosis: sarcoidosis
trichomoniasis: trichomoniasis
gonorrhea: nonrheumatic tricuspid valve disorders
b.cell acute lymphoblastic leukemia: acute sinusitis
chronic lymphocytic leukemia: monocytic leukemia
neoplasm of mature b.cells: other and unspecified diseases of blood and blood-forming organs
ewing sarcoma: trachoma
gastric carcinoma: carcinoma in situ of oral cavity, esophagus and stomach
hiv.1 infection: viral infection of unspecified site
head and neck squamous cell carcinoma: other and unspecified diseases of blood and blood-forming organs
hepatocellular carcinoma: carcinoma in situ of oral cavity, esophagus and stomach
hodgkins lymphoma: hodgkin lymphoma
invasive breast ductal carcinoma: carcinoma in situ of oral cavity, esophagus and stomach
malt lymphoma: other specified types of t/nk-cell lymphoma
metabolic syndrome: congenital iodine-deficiency syndrome
metastatic prostate

In [24]:
import pandas as pd
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

# Load the disease dataset
df_diseases = pd.read_csv("../CTD/data/target_disease/sample_cor_99.csv")
diseases = df_diseases.columns.tolist()

# Load the ICD-10 dataset
df_icd = pd.read_csv("data/DiagAll_Eng.csv")
df_icd = df_icd.drop_duplicates(subset=["ShortDescription"])  # Ensure unique descriptions
icd_dict = dict(zip(df_icd["ShortDescription"], df_icd["Code"]))
icd_descriptions = list(icd_dict.keys())
icd_codes = list(icd_dict.values())

# Load a pre-trained sentence transformer model
model = SentenceTransformer("NeuML/pubmedbert-base-embeddings")

print('Creating ICD10 embeddings...')
# Encode ICD-10 descriptions as embeddings without converting to tensor
icd_embeddings = model.encode(icd_descriptions, convert_to_tensor=False)

print('Precompute disease embeddings...')
disease_embeddings = {disease: model.encode(disease, convert_to_tensor=False) for disease in diseases}


/Users/vladimirk/opt/anaconda3/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Creating ICD10 embeddings...
Precompute disease embeddings...


In [25]:
def map_diseases(disease_name_1, disease_embedding, icd_dict, icd_descriptions, icd_codes, icd_embeddings):
#     disease_name_1_cleaned = disease_name_1.replace("_", " ").lower()
    
    # Check for exact match first
    if disease_name_1 in icd_dict:
        return disease_name_1
    
    # If no exact match, use semantic similarity
    disease_embedding_tensor = torch.tensor(disease_embedding)  # Ensure it's a tensor
    icd_embeddings_tensor = torch.tensor(icd_embeddings)  # Convert stored embeddings
    
    similarities = util.pytorch_cos_sim(disease_embedding_tensor, icd_embeddings_tensor)[0]
    best_match_idx = similarities.argmax().item()
    return icd_descriptions[best_match_idx] if similarities[best_match_idx] > 0.5 else "UNKNOWN"

# Map diseases to ICD-10 codes with progress tracking
diseases_mapping = {}
for disease in tqdm(diseases, desc="Mapping disease names to ICD-10"):
    diseases_mapping[disease] = map_diseases(disease, disease_embeddings[disease], icd_dict, icd_descriptions, icd_codes, icd_embeddings)

# Print a sample output
# Print each mapping on a new line
for disease, icd in list(diseases_mapping.items())[:50]:
    print(f"{disease}: {icd}")

Mapping disease names to ICD-10: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17701/17701 [00:05<00:00, 3448.60it/s]

synucleinopathy: Parkinson's disease
trypanosomiasis: African trypanosomiasis
giardiasis: Other protozoal intestinal diseases
pulmonary_sarcoidosis: Sarcoidosis
trichomoniasis: Trichomoniasis
gonorrhea: Gonococcal infection
B.cell_acute_lymphoblastic_leukemia: Lymphoid leukemia
chronic_lymphocytic_leukemia: Lymphoid leukemia
neoplasm_of_mature_B.cells: Mature T/NK-cell lymphomas
Ewing_sarcoma: UNKNOWN
gastric_carcinoma: Malignant neoplasm of stomach
HIV.1_infection: Human immunodeficiency virus [HIV] disease
head_and_neck_squamous_cell_carcinoma: Malignant neoplasm of base of tongue
hepatocellular_carcinoma: UNKNOWN
Hodgkins_lymphoma: Hodgkin lymphoma
invasive_breast_ductal_carcinoma: Carcinoma in situ of breast
MALT_lymphoma: Hodgkin lymphoma
metabolic_syndrome: Metabolic disorders
metastatic_prostate_cancer: Malignant neoplasm of prostate
mucinous_carcinoma: Malignant neoplasm of colon
myelodysplastic_syndrome: Myelodysplastic syndromes
oral_squamous_cell_carcinoma: Carcinoma in situ

In [28]:
import pandas as pd

# Assume corr_99_df is your large adjacency matrix and diseases_mapping is your dictionary
corr_99_df = pd.read_csv('../CTD/data/target_disease/cor_matrix_99.csv')
# Ensure the index is the same as columns
print("Setting index to match the columns...")
corr_99_df.index = corr_99_df.columns

# Filter columns based on diseases_mapping values that are not 'UNKNOWN'
print("Filtering columns based on valid diseases (not 'UNKNOWN')...")
valid_diseases = {k: v for k, v in diseases_mapping.items() if v != 'UNKNOWN'}

# Ensure only valid keys are used for filtering
valid_keys = set(valid_diseases.keys()) & set(corr_99_df.columns)
print(f"Selecting {len(valid_keys)} valid diseases from the matrix...")
filtered_corr_99_df = corr_99_df.loc[list(valid_keys), list(valid_keys)]

# Rename only the columns and rows that exist in the mapping
rename_mapping = {k: v for k, v in valid_diseases.items() if k in filtered_corr_99_df.columns}
print("Renaming rows and columns according to diseases_mapping...")
filtered_corr_99_df = filtered_corr_99_df.rename(index=rename_mapping, columns=rename_mapping)
print("Matrix shape:", filtered_corr_99_df.shape)
# Check for duplicated column names after renaming
# print("Checking for duplicated column names...")
duplicated_columns = filtered_corr_99_df.columns[filtered_corr_99_df.columns.duplicated()]

# Process each set of duplicated columns
if len(duplicated_columns) > 0:
#     print(f"Warning: Found duplicate column names after renaming: {duplicated_columns.tolist()}")
    
    # Loop through each duplicated column group
    for col in duplicated_columns.unique():
#         print(f"Processing duplicated columns for: {col}...")
        cols_to_mean = filtered_corr_99_df.loc[:, filtered_corr_99_df.columns == col]
        rows_to_mean = filtered_corr_99_df.loc[filtered_corr_99_df.index == col, :]

        if cols_to_mean.shape[1] > 1:
            # Compute the mean of the duplicated columns
#             print(f"Averaging the duplicated columns: {cols_to_mean.columns.tolist()}...")
            
            cols_to_mean = filtered_corr_99_df.loc[:, filtered_corr_99_df.columns == col]
            filtered_corr_99_df[col + '_temp'] = cols_to_mean.mean(axis=1)
            rows_to_mean = filtered_corr_99_df.loc[filtered_corr_99_df.index == col, :]
            filtered_corr_99_df.loc[col + '_temp', :] = rows_to_mean.mean(axis=0)
            filtered_corr_99_df = filtered_corr_99_df.drop(index=rows_to_mean.index[1:])
            filtered_corr_99_df = filtered_corr_99_df.drop(columns=cols_to_mean.columns[1:])
            filtered_corr_99_df.rename(index={col + '_temp' : col}, inplace=True)
            filtered_corr_99_df.rename(columns={col + '_temp' : col}, inplace=True)
            
            print("New matrix shape:", filtered_corr_99_df.shape)
                

# Display or save the transformed matrix
print("Transformation completed. New matrix shape:", filtered_corr_99_df.shape)  # To check the new dimensions


Setting index to match the columns...
Filtering columns based on valid diseases (not 'UNKNOWN')...
Selecting 9824 valid diseases from the matrix...
Renaming rows and columns according to diseases_mapping...
Matrix shape: (9824, 9824)
New matrix shape: (9563, 9563)
New matrix shape: (9503, 9503)
New matrix shape: (9410, 9410)
New matrix shape: (9346, 9346)
New matrix shape: (9313, 9313)
New matrix shape: (9295, 9295)
New matrix shape: (9145, 9145)
New matrix shape: (9070, 9070)
New matrix shape: (8949, 8949)
New matrix shape: (8761, 8761)
New matrix shape: (8741, 8741)
New matrix shape: (8610, 8610)
New matrix shape: (8600, 8600)
New matrix shape: (8538, 8538)
New matrix shape: (8499, 8499)
New matrix shape: (8467, 8467)
New matrix shape: (8442, 8442)
New matrix shape: (8425, 8425)
New matrix shape: (8376, 8376)
New matrix shape: (8371, 8371)
New matrix shape: (8307, 8307)
New matrix shape: (8302, 8302)
New matrix shape: (8292, 8292)
New matrix shape: (8269, 8269)
New matrix shape: (822

/var/folders/2h/45bqs84942sfxfwfndsxtgs40000gn/T/ipykernel_36479/1299389353.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  filtered_corr_99_df[col + '_temp'] = cols_to_mean.mean(axis=1)


New matrix shape: (5952, 5952)


/var/folders/2h/45bqs84942sfxfwfndsxtgs40000gn/T/ipykernel_36479/1299389353.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  filtered_corr_99_df[col + '_temp'] = cols_to_mean.mean(axis=1)


New matrix shape: (5939, 5939)


...

New matrix shape: (887, 887)


/var/folders/2h/45bqs84942sfxfwfndsxtgs40000gn/T/ipykernel_36479/1299389353.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  filtered_corr_99_df[col + '_temp'] = cols_to_mean.mean(axis=1)


New matrix shape: (886, 886)


/var/folders/2h/45bqs84942sfxfwfndsxtgs40000gn/T/ipykernel_36479/1299389353.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  filtered_corr_99_df[col + '_temp'] = cols_to_mean.mean(axis=1)


New matrix shape: (885, 885)
Transformation completed. New matrix shape: (885, 885)


In [29]:
# import pandas as pd
filtered_corr_99_df.to_csv('harmonized_open_target_corr_99.csv', index=False)
# filtered_corr_99_df = pd.read_csv('harmonized_open_target_corr_99.csv')

In [30]:
# Generate matrix with ICD10 codes
import pandas as pd
filtered_corr_99_df = pd.read_csv('harmonized_open_target_corr_99.csv')
# Load the ICD-10 dataset
df_icd = pd.read_csv("data/DiagAll_Eng.csv")
df_icd = df_icd.drop_duplicates(subset=["ShortDescription"])  # Ensure unique descriptions
icd_dict = dict(zip(df_icd["ShortDescription"], df_icd["Code"]))
filtered_corr_99_df.columns = [icd_dict[x] for x in filtered_corr_99_df.columns]
filtered_corr_99_df.to_csv('harmonized_open_target_corr_99_icd.csv', index=False)

In [16]:
df_icd[df_icd.duplicated('ShortDescription')]

,Code,ShortDescription
567,H03,Other disorders of eyelid
570,H06,Disorders of lacrimal system
573,H13,Other disorders of conjunctiva
597,H45,Disorders of vitreous body
609,H58,Other disorders of eye and adnexa
923,M03,Postinfective and reactive arthropathies


In [20]:
filtered_corr_99_df.columns

Index(['Sexual dysfunction not due to a substance or known physiological condition',
       'Unspecified protozoal disease', 'Varicose veins of other sites',
       'Disseminated intravascular coagulation [defibrination syndrome]',
       'Other and unspecified malignant neoplasms of lymphoid, hematopoietic and related tissue',
       'Simple and mucopurulent chronic bronchitis',
       'Nonrheumatic aortic valve disorders',
       'HIV disease with other specified diseases',
       'Mental disorder, not otherwise specified',
       'Acute posthemorrhagic anemia',
       ...
       'Echinococcosis_temp', 'Other nutritional deficiencies_temp',
       'Other disorders of breast_temp', 'Ascorbic acid deficiency_temp',
       'Other diseases of gallbladder_temp',
       'Other circulatory disorders_temp', 'Nonspecific lymphadenitis_temp',
       'Malignant neoplasm of other and unspecified female genital organs_temp',
       'Coalworker's pneumoconiosis_temp',
       'Other protozoal intes

In [65]:
filtered_corr_99_df.sum()

Sexual dysfunction not due to a substance or known physiological condition                  3.769266
Unspecified protozoal disease                                                               0.171629
Varicose veins of other sites                                                              11.599173
Disseminated intravascular coagulation [defibrination syndrome]                             3.501127
Other and unspecified malignant neoplasms of lymphoid, hematopoietic and related tissue     3.479422
                                                                                             ...    
Other circulatory disorders_temp                                                            7.144229
Nonspecific lymphadenitis_temp                                                              5.090659
Malignant neoplasm of other and unspecified female genital organs_temp                     10.677627
Coalworker's pneumoconiosis_temp                                                           

In [69]:
import numpy as np
np.count_nonzero(filtered_corr_99_df) / (885 * 885 )

0.3482779533339717